In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import StandardScaler

In [17]:
df_quake = pd.read_csv('data/quake.csv')
df_quake.columns = ['lat', 'long']
df_quake.head()

,lat,long
0,45.53,150.93
1,41.85,142.78
2,29.19,141.15
3,-21.66,169.81
4,23.09,120.58


A função a seguir repete a execução do KMeans $n$ vezes e retorna o modelo com o melhor erro de reconstrução, o qual pode ser calculado por:
$$
\mathcal{J}(\mathcal{C}) = \sum_{k = 1}^K \sum_{\boldsymbol{x}_i \in C_k} ||\boldsymbol{x} - \boldsymbol{m}_k||^2,
$$
Onde:
- $\boldsymbol{m}_k$ são os centróides do k-ésimo cluster;
- e $C_k$ são as partições.

OBS: o Scikit trata ele como inércia.

In [18]:
def run_kmeans_best_of_n(X, k, n_repeats=20, max_iter=300, random_state=0):
    best_model = None
    best_inertia = np.inf

    for r in range(n_repeats):
        km = KMeans(
            n_clusters=k,
            init='k-means++',
            n_init=1,              # controlamos a repetição fora do loop
            max_iter=max_iter,
            random_state=random_state,
            algorithm='lloyd'      # utiliza o algoritmo de Lloyd (padrão)
        )
        km.fit(X)
        if km.inertia_ < best_inertia:
            best_inertia = km.inertia_
            best_model = km
    return best_model, best_inertia

O índice Davies-Bouldin (DB) é computado como:
$$
\text{DB}(\mathcal{C}) = \frac{1}{K} \sum_{k = 1}^K \max_{k' \neq k} \left( \frac{\delta_k + \delta_{k'}}{\Delta_{kk'}} \right),
$$
Onde:
- $\delta_k$ é o espalhamento intra-agrupamento;
- e $\Delta_k$ é o  espalhamento entre grupos.

Além disso,
$$
\delta_k = \frac{1}{N_k} \sum_{\boldsymbol{x} \in C_k} ||\boldsymbol{x}_n - \boldsymbol{m}_k||,
$$
$$
\Delta_{kk'} = ||\boldsymbol{m}_k - \boldsymbol{m}_{k'}||,
$$
$$
\boldsymbol{m}_k = \frac{1}{N_k} \sum_{\boldsymbol{x} \in C_k} \boldsymbol{x}_i.
$$

In [19]:
def davies_bouldin(X, labels, centroids, dist_point_to_centroid, dist_centroid_to_centroid):
    k = centroids.shape[0]
    delta = np.zeros(k, dtype=float)

    # delta_k = (1/N_k) * Σ_{x in C_k} || x - m_k ||
    for kk in range(k):
        idx = np.where(labels == kk)[0]
        Nk = idx.size
        if Nk == 0:
            raise ValueError("Cluster vazio.")
        delta[kk] = np.mean([dist_point_to_centroid(X[i], centroids[kk]) for i in idx])


    # Delta_{k,k'} = || m_k - m_k' ||
    D = np.zeros((k, k), dtype=float)
    for i in range(k):
        for j in range(i + 1, k):
            d = dist_centroid_to_centroid(centroids[i], centroids[j])
            D[i, j] = D[j, i] = d


    # DB(C) = (1/K) * Σ_k max_{k'!=k} (delta_k + delta_k') / Delta_{k,k'}
    Rmax = np.zeros(k, dtype=float)
    for i in range(k):
        ratios = []
        for j in range(k):
            if i == j:
                continue
            if D[i, j] == 0:
                ratios.append(np.inf)
            else:
                ratios.append((delta[i] + delta[j]) / D[i, j])
        Rmax[i] = np.max(ratios)
    return np.mean(Rmax)

In [20]:
def plot_clusters(X, labels, centroids, title, xlabel="lon", ylabel="lat"):
    plt.figure(figsize=(7, 6))
    plt.scatter(X[:, 1], X[:, 0], c=labels, s=18, alpha=0.75)
    plt.scatter(centroids[:, 1], centroids[:, 0], c="black", s=120, marker="X")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, alpha=0.25)
    plt.show()

### Distância euclidiana

In [21]:
X_raw = df_quake
X = X_raw.copy()

scaler = StandardScaler()
X = scaler.fit_transform(X)

k_values = list(range(4, 21))
n_repeats = 20

results_euc = []
best_overall_euc = None  # (db, k, model, inercia)

for k in k_values:
    model, inertia = run_kmeans_best_of_n(X, k, n_repeats=n_repeats, random_state=12345)
    labels = model.labels_
    db = davies_bouldin_score(X, labels)  # DB com distancia euclidiana
    results_euc.append((k, db, inertia))

    if best_overall_euc is None or db < best_overall_euc[0]:
        best_overall_euc = (db, k, model, inertia)

results_euc = pd.DataFrame(results_euc, columns=["k", "db_euclid", "inertia"])
print("=== (a) Euclidiano: resultados por k ===")
print(results_euc.sort_values("db_euclid").head(10))
print("\nMelhor k (menor DB):", best_overall_euc[1], "DB:", best_overall_euc[0], "inertia:", best_overall_euc[3])

# Plot do melhor agrupamento (escala original)
best_db, best_k, best_model, best_inertia = best_overall_euc
labels_best = best_model.labels_
centroids_best = scaler.inverse_transform(best_model.cluster_centers_)
plot_clusters(
    X_raw,
    labels_best,
    centroids_best,
    title=f"(a) KMeans Euclidiano: melhor k={best_k} (DB={best_db:.4f})",
    xlabel=df_quake.columns[1],
    ylabel=df_quake.columns[0]
)

=== (a) Euclidiano: resultados por k ===
     k  db_euclid     inertia
15  19   0.583608   82.873131
6   10   0.591239  211.364562
16  20   0.601678   80.156561
13  17   0.610301  101.361647
14  18   0.612335   93.293470
9   13   0.613242  142.577391
10  14   0.613456  135.126374
0    4   0.613779  722.383603
12  16   0.615059  107.015062
11  15   0.615338  124.588887

Melhor k (menor DB): 19 DB: 0.5836078477069627 inertia: 82.87313063312217


InvalidIndexError: (slice(None, None, None), 1)

<Figure size 700x600 with 0 Axes>